# Análisis de datos exploratorio

## Cargar datos

In [2]:
import pandas as pd
import numpy as np
import os

In [7]:
df = pd.read_json("features.json", orient='records', lines=True)

In [52]:
test = pd.read_json("test.json", orient='records', lines=True)

## Inspeccionar datos

In [54]:
df.head(5)

,features,label
0,"[u, budget, fight, loom, republican, flip, fis...",1
1,"[u, military, accept, transgender, recruit, mo...",1
2,"[senior, u, republican, senator, let, mr, muel...",1
3,"[fbi, russia, probe, helped, australian, diplo...",1
4,"[trump, want, postal, service, charge, much, a...",1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180067 entries, 0 to 180066
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   features  180067 non-null  object
 1   label     180067 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.7+ MB


## Creacion de la representacion del texto (Word2Vec, Keras Embedding, BOW)

In [79]:
import gensim.downloader as api

# Descarga los vectores Word2Vec preentrenados en Google News (esto puede llevar tiempo)
word_vectors = api.load("word2vec-google-news-300")

In [80]:
from keras.preprocessing.text import Tokenizer


max_words = 1000
max_sequence_length = 100
embedding_dim = 300

tokenizer = Tokenizer(
    num_words=max_words
)  # max_words es la cantidad máxima de palabras que deseas considerar
tokenizer.fit_on_texts(df["features"])  # Ajusta el tokenizer a tus datos de texto
word_index = tokenizer.word_index

# Inicializa la matriz de embedding con los vectores preentrenados
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i >= max_words:
        break
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

del word_vectors

In [81]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

X_sequences = tokenizer.texts_to_sequences(df["features"]) 
T_sequences = tokenizer.texts_to_sequences(test["features"]) 

X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)
test_padded = pad_sequences(T_sequences, maxlen=max_sequence_length)

# X_train, X_test, y_train, y_test = train_test_split(
#     X_padded, df["label"], test_size=0.2, random_state=42
# )

X_train, y_train  = X_padded, df["label"]
X_test, y_test = test_padded, test["label"]

In [ ]:
tokenizer_json = tokenizer.index_word
inv_map = {v: k for k, v in tokenizer_json.items()}

In [ ]:
import json

# Nombre del archivo de salida
nombre_archivo = 'vocab.json'

# Guarda el vocabulario invertido en un archivo JSON
with open(nombre_archivo, 'w') as archivo:
    json.dump(inv_map, archivo)

print(f'Vocabulario invertido guardado en {nombre_archivo}')


Vocabulario invertido guardado en vocab.json


# Modelating

## Redes Neuronales

In [82]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.layers import Embedding, LSTM

In [83]:
# Construir y entrenar la red neuronal
model = Sequential()

# Capa de Embedding
model.add(
    Embedding(
    input_dim=max_words,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=max_sequence_length,
    trainable=False
    )
)
model.add(LSTM(units = 128, recurrent_dropout=0.25, dropout = 0.25))
model.add(Dense(1, activation = 'sigmoid' ))
model.compile(optimizer="adam", loss = 'binary_crossentropy', metrics=['accuracy'] )
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 100, 300)          300000    
                                                                 
 lstm_13 (LSTM)              (None, 128)               219648    
                                                                 
 dense_14 (Dense)            (None, 1)                 129       
                                                                 
Total params: 519777 (1.98 MB)
Trainable params: 219777 (858.50 KB)
Non-trainable params: 300000 (1.14 MB)
_________________________________________________________________


In [84]:
history = model.fit(
    np.array(X_train),
    y_train,
    epochs=25,
    batch_size=32,
    verbose=True,
    validation_data=(np.array(X_test), y_test),
    workers=4,
)

Epoch 1/25
1355/2813 [=============>................] - ETA: 4:24 - loss: 0.2023 - accuracy: 0.9240

KeyboardInterrupt: 

In [ ]:
model.save("modelo.h5")

In [ ]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import string

In [ ]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text
lemmatizer = WordNetLemmatizer()

In [ ]:
def process_text(text):
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    text = re.sub(r'\$\w*', '', text)
    # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    # remove hashtags
    text = re.sub(r'#', '', text)
    text = str(re.sub("\S*\d\S*", "", text).strip())
    text = decontract(text)

    # tokenize texts
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tokens = tokenizer.tokenize(text)

    texts_clean = []
    for word in tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation+'...'):  # remove punctuation
            #
            stem_word = lemmatizer.lemmatize(word,"v")  # Lemmatizing word
            texts_clean.append(stem_word)

    return " ".join(texts_clean)

In [ ]:
%pip install lime

In [ ]:
from lime.lime_text import LimeTextExplainer
class_names=['NotFake','IsFake']
explainer= LimeTextExplainer(class_names=class_names)

def predict_proba(arr):
  processed=[]
  for i in arr:
    processed.append(process_text(i))
  list_tokenized_ex = tokenizer.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_sequence_length)
  pred=model.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  return np.array(returnable)

In [ ]:
print("Actual rating",df['label'][2])
explainer.explain_instance(df['text'][2],predict_proba).show_in_notebook(text=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Bolsa de palabras
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(df['features'][0])

palabras = vectorizer.get_feature_names_out()

frecuencias = texto_features.toarray()

In [ ]:
frecuencias_totales = frecuencias.sum(axis=0)

# Obtener las palabras más frecuentes y sus frecuencias
palabras_mas_frecuentes = [palabras[i] for i in frecuencias_totales.argsort()[::-1][:10]]
frecuencias_mas_frecuentes = [frecuencias_totales[i] for i in frecuencias_totales.argsort()[::-1][:10]]

# Crear un gráfico de barras horizontal
plt.figure(figsize=(10, 6))
plt.barh(palabras_mas_frecuentes, frecuencias_mas_frecuentes, color='skyblue')
plt.xlabel('Frecuencia')
plt.ylabel('Palabra')
plt.title('Palabras más frecuentes')
plt.gca().invert_yaxis()  # Invertir el eje y para mostrar las palabras más frecuentes arriba
plt.show()

## Evaluation

In [ ]:
# Evaluar el modelo
loss, accuracy = model.evaluate(np.array(X_test), y_test)
print(f'Accuracy: {accuracy}')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
predictions = model.predict(np.array(X_test))
predictions = list(map(lambda x: 1 if (x > 0.5) else 0, predictions))

cm = confusion_matrix(y_test, predictions,labels=[0,1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot()

In [ ]:
print(classification_report(y_test, list(predictions), digits=4))

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

In [ ]:
%pip install skl2onnx
%pip install tensorflowjs

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "rnn_model")